In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from imutils import paths
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img
from tensorflow.keras.layers import Reshape 
from keras.models import Model, load_model
from keras.layers import Input
from tensorflow.keras.utils import plot_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten

making first input layer

In [ ]:
#inputt = Input(shape=(10,))

making output of x we writing this node as frist its output name than in last 

we will write the value from which this node is taking output

In [ ]:
#hidden1 = Dense(128, activation="relu")(inputt)
#hidden2 = Dense(64, activation="relu")(hidden1)

Now from this 2nd dense/hidden2 layer we will make a branch or split this into two part output1 and output2

In [ ]:
#output1 = Dense(1, activation="sigmoid")(hidden2)
#output2 = Dense(5, activation="softmax")(hidden2)

In [ ]:
#my_model=Model(inputs= inputt,outputs=[output1,output2])

In [ ]:
#my_model.summary()

You can also plot the model as a graph: and show_shapes=True used to show input and output

In [ ]:
#plot_model(model, "my_model_with_shape_info.png", show_shapes=True)

In [ ]:
# small data
#image_path = r'C:\Users\sonuc\Desktop\Data_Science\Transpher learning and cat-and-dog SIMPLE\Transpher_learning\Method_3\utk_facee\crop_part1'

In [ ]:
# big size data
image_path = r'C:\Users\sonuc\Desktop\Data_Science\Transpher learning and cat-and-dog SIMPLE\Transpher_learning\Method_3\utk_facee\UTKFace'

joining path between dog,cat folder and i by using path.join

listing folders using listdir

In [ ]:
Age = []
gender = []
imgg = []


for imagess in os.listdir(image_path):
    
    img_path = os.path.join(image_path,imagess)
    Age.append(int(imagess.split('_')[0]))
    gender.append(int(imagess.split('_')[1]))
    
    img = image.load_img(img_path,target_size=(100, 100))
    img = image.img_to_array(img)
    #img = img.reshape (1,200,200,3)
    imgg.append(img)
   

In [ ]:
Age

In [ ]:
x = np.array(imgg)
x = x/255.0

In [ ]:
labels = []
for i in range(len(Age)):
    
    label = [Age[i],gender[i]]
    labels.append(label)

In [ ]:
y = np.array(labels)

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
y[0]

as we can see both age and gender in same list  but we need both in sepreate list

In [ ]:
y = [y[:,0],y[:,1]]
#y_train = [y_train[:,0],y_train[:,1]]

In [ ]:
resnet = ResNet50(include_top = False,input_shape = (100,100,3))

In [ ]:
resnet.summary()

connecting a flatten layer to resnet last layer

In [ ]:
output = resnet.layers[-1].output
flatten = Flatten()(output)

Now from this flatten layer we will make a branch or split this into two part output1 and output2

In [ ]:
dense1 = Dense(512, activation="relu")(flatten)
dense2 = Dense(512, activation="relu")(flatten)

dense3 = Dense(512, activation="relu")(dense1)
dense4 = Dense(512, activation="relu")(dense2)

making output layer for classification (gender) and regression (age)

In [ ]:
output_regression = Dense(1, activation="linear")(dense3)
output_classification = Dense(1, activation="sigmoid")(dense4)

In [ ]:
resnet_model=Model(inputs= resnet.input,outputs=[output_regression,output_classification])

Plotinng model

In [ ]:
#plot_model(resnet_model, "resnet_model_with_shape_info.png", show_shapes=True)

In [ ]:
# freezing all layers of resnet50 model
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
#checking trainable parameter
resnet_model.summary()

In [ ]:
resnet_model.compile(loss= ['binary_crossentropy', 'mae'],
              optimizer='Adam',
              metrics=['accuracy', 'mae'])

In [ ]:
History = resnet_model.fit(x,y, epochs=100, validation_split=0.2 ,batch_size = 32)

In [ ]:
resnet_model.save('gender_age.h5')  # always save your weights after training or during training

In [ ]:
model = load_model('gender_age.h5')

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'MAN',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [ ]:
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()